In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dropout,AveragePooling2D, merge, Activation,SeparableConv2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20

Using TensorFlow backend.


In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 20s 0us/step


In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=8)

In [0]:
input = Input(shape=(img_height, img_width, channel,))

# Layer 1
x = SeparableConv2D(32, (5,5),border_mode='same', name='Sep_conv_1')(input)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_1')(x)

layer1 = x

# Layer 2
x = Conv2D(64, (5,5), name='Conv_1',border_mode='same')(x)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_2')(x)

layer2 = x

# Layer 3
x = Conv2D(128, (5,5), name='Conv_2', border_mode='same')(x)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_3')(x)

layer3 = x

skip1 = concatenate([layer1, layer3])

# Layer 4
x = SeparableConv2D(256, (5,5), name='Sep_conv_2',border_mode='same')(skip1)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_4')(x)

layer4 = x

# Layer 5
skip2 = concatenate([layer1, layer4])

x = MaxPooling2D(pool_size=(2, 2))(skip2)


# Layer 6
x = SeparableConv2D(32, (3,3), border_mode='same', name='Sep_conv_3', use_bias=False)(x)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_6')(x)

# Layer 7
x = Conv2D(64, (5,5), name='Conv_3', border_mode='same')(x)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_7')(x)

# Layer 8
x = SeparableConv2D(128, (3,3), border_mode='same', name='Sep_conv_4', use_bias=False)(x)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_8')(x)


# Layer 9
x = SeparableConv2D(256, (5,5), border_mode='same', name='Sep_conv_5', use_bias=False)(x)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_9')(x)

# Layer 10
skip2 = concatenate([layer1, layer4])

x = MaxPooling2D(pool_size=(2, 2))(skip2)

# Layer 11
x = Conv2D(32, (5,5), name='Conv_4',border_mode='same')(skip1)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_11')(x)

# Layer 12
x = SeparableConv2D(64, (5,5), border_mode='same', name='Sep_conv_5', use_bias=False)(x)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_12')(x)

# Layer 13
x = Conv2D(128, (3,3), name='Conv_5',border_mode='same',)(x)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_13')(x)

# Layer 14
x = SeparableConv2D(64, (5,5), border_mode='same', name='Sep_conv_5', use_bias=False)(x)
x = Activation('relu'))(x)
x = BatchNormalization(name='norm_14')(x)

x = concatenate([block_4, block_1])

# Layer 23
x = Flatten()(x)

output = Activation('softmax')(x)



In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 32, 32, 32)   864         input_8[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 32, 32, 32)   128         conv_1[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu_141 (LeakyReLU)     (None, 32, 32, 32)   0           norm_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Co

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 254s 5ms/step - loss: 11.6765 - acc: 0.2378 - val_loss: 12.4499 - val_acc: 0.2056
Epoch 2/50
50000/50000 [==============================] - 232s 5ms/step - loss: 5.8984 - acc: 0.3242 - val_loss: 10.7627 - val_acc: 0.2002
Epoch 3/50
50000/50000 [==============================] - 233s 5ms/step - loss: 2.3266 - acc: 0.4774 - val_loss: 2.2493 - val_acc: 0.4199
Epoch 4/50
50000/50000 [==============================] - 233s 5ms/step - loss: 1.8994 - acc: 0.5393 - val_loss: 3.3840 - val_acc: 0.3643
Epoch 5/50
50000/50000 [==============================] - 233s 5ms/step - loss: 1.6164 - acc: 0.5931 - val_loss: 3.6200 - val_acc: 0.3418
Epoch 6/50
50000/50000 [==============================] - 232s 5ms/step - loss: 1.4143 - acc: 0.6289 - val_loss: 1.4980 - val_acc: 0.5832
Epoch 7/50
50000/50000 [==============================] - 232s 5ms/step

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 18s 2ms/step
Test loss: 2.3465495806694032
Test accuracy: 0.7064
Saved the model to disk


In [0]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model.h5')

Saved the model to disk


FileNotFoundError: ignored